<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Interfaces" data-toc-modified-id="Interfaces-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Interfaces</a></span><ul class="toc-item"><li><span><a href="#Interfaces-as-Contracts" data-toc-modified-id="Interfaces-as-Contracts-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Interfaces as Contracts</a></span></li><li><span><a href="#Interface-Types" data-toc-modified-id="Interface-Types-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Interface Types</a></span></li><li><span><a href="#Interface-Satisfaction" data-toc-modified-id="Interface-Satisfaction-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Interface Satisfaction</a></span></li><li><span><a href="#Parsing-Flags-with-flag.Value" data-toc-modified-id="Parsing-Flags-with-flag.Value-7.4"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>Parsing Flags with flag.Value</a></span></li><li><span><a href="#Interface-Values" data-toc-modified-id="Interface-Values-7.5"><span class="toc-item-num">7.5&nbsp;&nbsp;</span>Interface Values</a></span></li><li><span><a href="#Sorting-with-sort.Interface" data-toc-modified-id="Sorting-with-sort.Interface-7.6"><span class="toc-item-num">7.6&nbsp;&nbsp;</span>Sorting with sort.Interface</a></span></li><li><span><a href="#The-http.Handler-Interface" data-toc-modified-id="The-http.Handler-Interface-7.7"><span class="toc-item-num">7.7&nbsp;&nbsp;</span>The http.Handler Interface</a></span></li><li><span><a href="#The-error-Interface" data-toc-modified-id="The-error-Interface-7.8"><span class="toc-item-num">7.8&nbsp;&nbsp;</span>The error Interface</a></span></li><li><span><a href="#Example:-Expression-Evaluator" data-toc-modified-id="Example:-Expression-Evaluator-7.9"><span class="toc-item-num">7.9&nbsp;&nbsp;</span>Example: Expression Evaluator</a></span></li><li><span><a href="#Type-Assertions" data-toc-modified-id="Type-Assertions-7.10"><span class="toc-item-num">7.10&nbsp;&nbsp;</span>Type Assertions</a></span></li><li><span><a href="#Discriminating-Errors-with-Type-Assertions" data-toc-modified-id="Discriminating-Errors-with-Type-Assertions-7.11"><span class="toc-item-num">7.11&nbsp;&nbsp;</span>Discriminating Errors with Type Assertions</a></span></li><li><span><a href="#Querying-Behaviors-with-Interface-Type-Assertions" data-toc-modified-id="Querying-Behaviors-with-Interface-Type-Assertions-7.12"><span class="toc-item-num">7.12&nbsp;&nbsp;</span>Querying Behaviors with Interface Type Assertions</a></span></li><li><span><a href="#Type-Switches" data-toc-modified-id="Type-Switches-7.13"><span class="toc-item-num">7.13&nbsp;&nbsp;</span>Type Switches</a></span></li><li><span><a href="#Example:-Token-Based-XML-Decoding" data-toc-modified-id="Example:-Token-Based-XML-Decoding-7.14"><span class="toc-item-num">7.14&nbsp;&nbsp;</span>Example: Token-Based XML Decoding</a></span></li></ul></li></ul></div>

# Interfaces

## Interfaces as Contracts

* Concrete type specifies the exact representation of its values and exposes the intrinsic operations of tht representation
* Interface is an abstract type

In [105]:
//package fmt
import ("io"; "os";"bytes")
func Fprintf(w io.Writer, format string, args ...interface{})(int, error)
func Printf(format string, args ...interface{})(int, error) {
    return fmt.Fprintf(os.Stdout, format, args...)
}
Printf("%s,%d\n","test",100)

test,100


9 <nil>

* Writer is the interface that wraps the basic Write method
* Write writes len(p) bytes from p to the underlying data stream.
* Returns the number of bytes from p (0 $\le$ n $\le$ len(p))
* Write must return a non-nil error if returns n < len(p)
* Write must not modify the slice data, even temporarily

In [106]:
//package io
//Writer is the interface that wraps the basic Write method
type Writer interface {
    Write(p []byte) (n int, err error)
}

In [20]:
func Sprintf(format string, args ...interface{}) string {
    var buf bytes.Buffer
    fmt.Fprintf(&buf, format, args...)
    return buf.String()
}
Sprintf("%s,%d","test",100)

test,100

In [108]:
//gopl.io/ch7/bytecounter
import "fmt"
type ByteCounter struct {i int}
func (c *ByteCounter) Write(p []byte) (int, error) {
    c.i += len(p) // convert int to ByteCounter
    return len(p), nil
}
var d ByteCounter
c := ByteCounter{0}
c.Write([]byte("hello"))
fmt.Println(c)

{5}


4 <nil>

In [109]:
// Bytecounter demonstrates an implementation of io.Writer that counts bytes.
//package main

import ("fmt")

//!+bytecounter

type ByteCounter struct {i int}

func (c *ByteCounter) Write(p []byte) (int, error) {
	c.i += (len(p)) // convert int to ByteCounter
	return len(p), nil
}

//!-bytecounter

func main() {
	//!+main
    c := ByteCounter{0}
    c.Write([]byte("hello"))
	fmt.Println(c) // "5", = len("hello")

	c.i = 0 // reset the counter
	var name = "Dolly"
	fmt.Fprintf(&c, "hello, %s", name)
	fmt.Println(c) // "12", = len("hello, Dolly")
	//!-main
}
main()

ERROR: repl.go:25:31: cannot convert type <*main.ByteCounter> to interface <io.Writer>: missing method  Write

## Interface Types
* An interface type specifies a set of metods that a concrete type must possess to be considered an instance of that interface 

In [110]:
//package io
type Reader interface {
    Read(p []byte) (n int, err error)
}
type Closer interface {
    Close() error
}


In [111]:
type ReadWriter interface {
    Reader
    Writer
}
type ReadWriteCloser interface {
    Reader
    Writer
    Closer
}

ERROR: go/parser internal error: identifier already declared or resolved

In [112]:
type ReadWriter interface {
    Read(p []byte) (n int, err error)
    Write(p []byte) (n int, err error)
}

In [113]:
type ReadWriter interface {
    Read(p []byte) (n int, err error)
    Writer
}

ERROR: go/parser internal error: identifier already declared or resolved

## Interface Satisfaction
* A type satisfies an interface if it possesses all the methods the interface requires
* An expression may be assigned to an interface only if its type satisfies the interface

In [114]:
import "time"
var w io.Writer
w = os.Stdout        //OK: *os.File has Write method
w = new(bytes.Buffer)//OK, *bytes.Buffeer has Write method 


In [115]:
w = time.Second // compile error: time.Duration lacks Write method

ERROR: repl.go:1:1: error compiling assignment: w = time.Second
	cannot convert typed constant 1s <time.Duration> to <io.Writer>
	reason: time.Duration does not implement io.Writer: missing method func (interface{Write(p []uint8) (n int, err error)}).Write(p []uint8) (n int, err error)

In [97]:
var rwc io.ReadWriteCloser
rwc = os.Stdout    //OK: *os.File has Read,Write, Close methods

In [98]:
rwc = new(bytes.Buffer)// compile error: *bytes.Buffer lacks Close method

ERROR: repl.go:1:1: error compiling assignment: rwc = new(bytes.Buffer)
	repl.go:1:1: incompatible types in assignment: io.ReadWriteCloser = *bytes.Buffer
	reason: *bytes.Buffer does not implement io.ReadWriteCloser: missing method func (interface{Close() error}).Close() error

In [99]:
w = rwc  // OK: io.ReadWriteCloser has Write method

In [100]:
rwc = w // compile error: io.Writer lacks Close metho

ERROR: repl.go:1:1: error compiling assignment: rwc = w
	repl.go:1:1: incompatible types in assignment: io.ReadWriteCloser = io.Writer
	reason: io.Writer does not implement io.ReadWriteCloser: missing method func (interface{Close() error}).Close() error

In [119]:
type IntSet struct {/* ... */}
func (*IntSet) String() string
var _ = IntSet{}.String()

In [122]:
var s IntSet
var _ = s.String() // OK: s is a variable and &s has a String method

In [129]:
var _ fmt.Stringer = &s // OK, fmt.Stringer is an interface
var _ fmt.Stringer = s 

In [131]:
os.Stdout.Write([]byte("hello")) // OK: *os.File has Write method
os.Stdout.Close()                // OK: *os.File has Close method
var w io.Writer
w = os.Stdout
w.Write([]byte("hello"))         // OK: io.Writer has Write method
w.Close()                        // compile error:io.Writer lacks Close method

ERROR: repl.go:6:1: not a package: "w" in w.Close <*ast.SelectorExpr>

* the type interface{} has no methods 
* the empty interface type places no demands on the types that satisfy it
* We can assign any value to the empty interface
* fmt.Println or errorf to accept arguments of any type

In [140]:
var any interface{}
any = true
any = 12.34
any = "hello"
any = map[string]int{"one":1}
any = new(bytes.Buffer)

In [142]:
// *bytes.Buffer must satisfy io.Writer
var w io.Writer = new(bytes.Buffer)
var _ io.Writer = (*bytes.Buffer)(nil)

* A concrete type may satisfy many unrelated interfaces
* We may define the following set of concrete types:
  * Album
  * Book
  * Movie
  * Magazine
  * Podcast
  * TVEpisode
  * Track

* We can express each abstration of interest as ann interface
* Some properties are common to all artifacts, such as title, a creation date, and a list of 

In [ ]:
type Artifact interface {
    Title() string
    Creators() []string
    Created() time.Time
}

In [146]:
type Text interface {
    Pages() int
    Words() int
    PageSize() int
}
type Audio interface {
    Stream() (io.ReadCloser, error)
    RuningTime() time.Duration
    Format() string // e.g., "MP3", "WAV"
} 
type Video interface {
    Stream() (io.ReadCloser, error)
    RuningTime() time.Duration
    Format() string // e.g., "MP4", "WMV"
    Resolution() (x,y int)
}

* Each grouping of concrete ypes based on the shared behaviors can be expressed as an interface type
* Set of interfaces satisfied by a class in class-based languages are explicit
* In Golang, we can define new abstractions or groupings of interest when we need them, without modifying the declaration of the concrete type

In [145]:
type Streamer interface {
    Stream() (io.ReadCloser, error)
    RunningTime() time.Duration
    Format() string
}

## Parsing Flags with flag.Value

In [7]:
//gopl.io/ch7/sleep
import ("time";"flag";"fmt";"os")
var period = flag.Duration("period", 1*time.Second, "sleep period")
func main() {
    flag.Parse()
    fmt.Printf("Sleeping for %v...", *period)
    time.Sleep(*period)
    fmt.Println()
}
os.Args = []string{"sleep","-period","50ms"}
main()
os.Args = []string{"sleep","-period","2m30s"}
main()
os.Args = []string{"sleep","-period","1 day"}
main()

gophernotes flag redefined: period


ERROR: gophernotes flag redefined: period

<code>
    package flag
    //Value is the interface to the value stored in a flag
    type Value interface {
      String() string
      Set(string) error
    }
</code>

In [11]:
//gopl.io/ch7/tempconv
// *celsiusFlag satisfies the flag.Value interface.
import (
	"flag"
	"fmt"
)

type Celsius float64
type Fahrenheit float64

func CToF(c Celsius) Fahrenheit { return Fahrenheit(c*9.0/5.0 + 32.0) }
func FToC(f Fahrenheit) Celsius { return Celsius((f - 32.0) * 5.0 / 9.0) }

func (c Celsius) String() string { return fmt.Sprintf("%g°C", c) }
type celsiusFlag struct {Celsius}
func (f *celsiusFlag) Set(s string) error {
    var unit string
    var value float64
    fmt.Sscanf(s, "%f%s", &value, &unit) // no error check needed
    switch unit {
    case "C", "°C":
        f.Celsius = Celsius(value)
        return nil
    case "F", "°F":
        f.Celsius = FToC(Fahrenheit(value))
        return nil
    }
    return fmt.Errorf("invalid temperature %q",s)
}
func CelsiusFlag(name string, value Celsius, usage string) *Celsius {
	f := celsiusFlag{value}
	flag.CommandLine.Var(&f, name, usage)
	return &f.Celsius
}

ERROR: repl.go:32:33: cannot convert type <*main.celsiusFlag> to interface <flag.Value>: missing method  Set

## Interface Values

In [15]:
import "io"
var w io.Writer
fmt.Printf("%v, %T\n", w,w)

<nil>, <nil>


13 <nil>

In [17]:
w = os.Stdout
fmt.Printf("%v, %T\n", w,w)

&{0xc0006376e0}, *os.File


26 <nil>

In [19]:
import "bytes"
w = new(bytes.Buffer)
fmt.Printf("%v, %T\n",w,w)

, *bytes.Buffer


16 <nil>

## Sorting with sort.Interface

In [4]:
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 187.

// Sorting sorts a music playlist into a variety of orders.
//package main

import (
	"fmt"
	"os"
	"sort"
	"text/tabwriter"
	"time"
)

//!+main
type Track struct {
	Title  string
	Artist string
	Album  string
	Year   int
	Length time.Duration
}

var tracks = []*Track{
	{"Go", "Delilah", "From the Roots Up", 2012, length("3m38s")},
	{"Go", "Moby", "Moby", 1992, length("3m37s")},
	{"Go Ahead", "Alicia Keys", "As I Am", 2007, length("4m36s")},
	{"Ready 2 Go", "Martin Solveig", "Smash", 2011, length("4m24s")},
}

func length(s string) time.Duration {
	d, err := time.ParseDuration(s)
	if err != nil {
		panic(s)
	}
	return d
}


In [5]:
//!-main

//!+printTracks
func printTracks(tracks []*Track) {
	const format = "%v\t%v\t%v\t%v\t%v\t\n"
	tw := new(tabwriter.Writer).Init(os.Stdout, 0, 8, 2, ' ', 0)
	fmt.Fprintf(tw, format, "Title", "Artist", "Album", "Year", "Length")
	fmt.Fprintf(tw, format, "-----", "------", "-----", "----", "------")
	for _, t := range tracks {
		fmt.Fprintf(tw, format, t.Title, t.Artist, t.Album, t.Year, t.Length)
	}
	tw.Flush() // calculate column widths and print table
}

//!-printTracks



In [6]:
//!+artistcode
type byArtist []*Track

func (x byArtist) Len() int           { return len(x) }
func (x byArtist) Less(i, j int) bool { return x[i].Artist < x[j].Artist }
func (x byArtist) Swap(i, j int)      { x[i], x[j] = x[j], x[i] }

//!-artistcode

//!+yearcode
type byYear []*Track

func (x byYear) Len() int           { return len(x) }
func (x byYear) Less(i, j int) bool { return x[i].Year < x[j].Year }
func (x byYear) Swap(i, j int)      { x[i], x[j] = x[j], x[i] }

//!-yearcode

    fmt.Println("byArtist:")
	sort.Sort(byArtist(tracks))
	printTracks(tracks)

	fmt.Println("\nReverse(byArtist):")
	sort.Sort(sort.Reverse(byArtist(tracks)))
	printTracks(tracks)

	fmt.Println("\nbyYear:")
	sort.Sort(byYear(tracks))
	printTracks(tracks)


byArtist:
Title       Artist          Album              Year  Length  
-----       ------          -----              ----  ------  
Go Ahead    Alicia Keys     As I Am            2007  4m36s   
Go          Delilah         From the Roots Up  2012  3m38s   
Ready 2 Go  Martin Solveig  Smash              2011  4m24s   
Go          Moby            Moby               1992  3m37s   

Reverse(byArtist):
Title       Artist          Album              Year  Length  
-----       ------          -----              ----  ------  
Go          Moby            Moby               1992  3m37s   
Ready 2 Go  Martin Solveig  Smash              2011  4m24s   
Go          Delilah         From the Roots Up  2012  3m38s   
Go Ahead    Alicia Keys     As I Am            2007  4m36s   

byYear:
Title       Artist          Album              Year  Length  
-----       ------          -----              ----  ------  
Go          Moby            Moby               1992  3m37s   
Go Ahead    Alicia Keys     As 

In [7]:
func main() {
	
	fmt.Println("\nCustom:")
	//!+customcall
	sort.Sort(customSort{tracks, func(x, y *Track) bool {
		if x.Title != y.Title {
			return x.Title < y.Title
		}
		if x.Year != y.Year {
			return x.Year < y.Year
		}
		if x.Length != y.Length {
			return x.Length < y.Length
		}
		return false
	}})
	//!-customcall
	printTracks(tracks)
}

/*
//!+artistoutput
Title       Artist          Album              Year  Length
-----       ------          -----              ----  ------
Go Ahead    Alicia Keys     As I Am            2007  4m36s
Go          Delilah         From the Roots Up  2012  3m38s
Ready 2 Go  Martin Solveig  Smash              2011  4m24s
Go          Moby            Moby               1992  3m37s
//!-artistoutput

//!+artistrevoutput
Title       Artist          Album              Year  Length
-----       ------          -----              ----  ------
Go          Moby            Moby               1992  3m37s
Ready 2 Go  Martin Solveig  Smash              2011  4m24s
Go          Delilah         From the Roots Up  2012  3m38s
Go Ahead    Alicia Keys     As I Am            2007  4m36s
//!-artistrevoutput

//!+yearoutput
Title       Artist          Album              Year  Length
-----       ------          -----              ----  ------
Go          Moby            Moby               1992  3m37s
Go Ahead    Alicia Keys     As I Am            2007  4m36s
Ready 2 Go  Martin Solveig  Smash              2011  4m24s
Go          Delilah         From the Roots Up  2012  3m38s
//!-yearoutput

//!+customout
Title       Artist          Album              Year  Length
-----       ------          -----              ----  ------
Go          Moby            Moby               1992  3m37s
Go          Delilah         From the Roots Up  2012  3m38s
Go Ahead    Alicia Keys     As I Am            2007  4m36s
Ready 2 Go  Martin Solveig  Smash              2011  4m24s
//!-customout
*/

//!+customcode
type customSort struct {
	t    []*Track
	less func(x, y *Track) bool
}

func (x customSort) Len() int           { return len(x.t) }
func (x customSort) Less(i, j int) bool { return x.less(x.t[i], x.t[j]) }
func (x customSort) Swap(i, j int)      { x.t[i], x.t[j] = x.t[j], x.t[i] }

//!-customcode

func init() {
	//!+ints
	values := []int{3, 1, 4, 1}
	fmt.Println(sort.IntsAreSorted(values)) // "false"
	sort.Ints(values)
	fmt.Println(values)                     // "[1 1 3 4]"
	fmt.Println(sort.IntsAreSorted(values)) // "true"
	sort.Sort(sort.Reverse(sort.IntSlice(values)))
	fmt.Println(values)                     // "[4 3 1 1]"
	fmt.Println(sort.IntsAreSorted(values)) // "false"
	//!-ints
}

ERROR: repl.go:15:10: cannot use <main.customSort> as <sort.Interface> in argument to sort.Sort

## The http.Handler Interface

In [8]:
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 191.

// Http1 is a rudimentary e-commerce server.
//package main

import (
	"fmt"
	"log"
	"net/http"
)

//!+main

func main() {
	db := database{"shoes": 50, "socks": 5}
	log.Fatal(http.ListenAndServe("localhost:8000", db))
}

type dollars float32

func (d dollars) String() string { return fmt.Sprintf("$%.2f", d) }

type database map[string]dollars

func (db database) ServeHTTP(w http.ResponseWriter, req *http.Request) {
	for item, price := range db {
		fmt.Fprintf(w, "%s: %s\n", item, price)
	}
}

//!-main

/*
//!+handler
package http

type Handler interface {
	ServeHTTP(w ResponseWriter, r *Request)
}

func ListenAndServe(address string, h Handler) error
//!-handler
*/

ERROR: repl.go:19:50: cannot use <main.database> as <net/http.Handler> in argument to http.ListenAndServe

## The error Interface

## Example: Expression Evaluator

## Type Assertions

## Discriminating Errors with Type Assertions

## Querying Behaviors with Interface Type Assertions

## Type Switches

## Example: Token-Based XML Decoding